<a href="https://www.kaggle.com/code/rikuishiharaa/mun-poi-point-of-interest-generation?scriptVersionId=247400029" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# What is MUN?

MUN, short for Model United Nations is a club that replicates the procedure of the United Nations. Students represent a country and debate on a topic to create resolutions. 

# What are POIs?

POIs, short for Point of Interests are questions asked at the end of the speech. To help delegates answer such questions eloquently, I utilized prompt engineering to generate prompts.

 
 ![image](https://3.files.edl.io/d284/19/11/24/011439-72dcd7d2-2e74-4f2b-8907-ef18254c80b5.jpg) 

In [1]:
from openai import AzureOpenAI
import os
from openai import OpenAI

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
api = user_secrets.get_secret("OPENAI_API_KEY")
api_2 = user_secrets.get_secret("OPENAI_API_KEY_2")

client_2 = OpenAI(api_key=api_2)

client = AzureOpenAI(
    api_key=api,
    api_version="2024-11-01-preview",
    azure_endpoint="https://swedencentral.api.cognitive.microsoft.com"
)

In [2]:
import pandas as pd

data = pd.read_csv("/kaggle/input/mun-poi-speech-dataset/MUNSPEECHPOI.csv", on_bad_lines='skip')
speeches = pd.read_csv("/kaggle/input/mun-speech-dataset/MUN Speech Testing.csv")

In [3]:
chain_of_thought = "First, generate 3 opposing points from 3 different perspective countries. One of the several points made by the speech, choose 1 and formualte a point opposing that. The POI shouldn't be a general POI about the entire speech. Then, convert that into POIs in the form of a question."

# Prompt Engineering

## Original Functions

In [4]:
# def make_poi_few(speech, model, country):
#     prompt = "Use these POI and Speech pairs to predict POIs for a speech."
#     for index, row in data.iterrows():
#         prompt = prompt + " Data number: "+str(index)+" Speech: "+row['speech']+" POI: "+row['poi']+"Country: "+row['country_of_speech']
#     prompt += "Word limit of 20 words.  The output format should be 1. POI1\n2. POI2\n3. POI3"
#     prompt += "\n\n\n\n Now predict the POI for this speech: " + speech
#     prompt += "With this country: " + country
#     prompt += "Know that when predicting a POI, I only want you to give the POI and nothing else. The POI should be in the style of the dataset that I provided. Don't include anything specific to this commitee, such as things about russia when there is nothing about russia in the speech. Just match the style and the general patterns of the POI. Give me 3 POIs. "
    
#     response = client.chat.completions.create(
#         model=model,
#         messages=[
#             {"role": "user", "content": prompt}
#         ]
#     )
#     print(response.choices[0].message.content)

# def make_poi_zero(speech, model, country):
#     prompt = "Generate POIs for the MUN speech. Give me only the actual POI and nothing else for the output. Give me 3 POIs. "
#     prompt += "Give me the POI for this speech: " + speech
#     prompt += "With this country: " + country
#     prompt += "Word limit of 20 words.  The output format should be 1. POI1\n2. POI2\n3. POI3"
    
#     response = client.chat.completions.create(
#         model=model,
#         messages=[
#             {"role": "user", "content": prompt}
#         ]
#     )
    
#     print(response.choices[0].message.content)

# def make_poi_few_chain(speech, model, country):
#     prompt = "Use these POI and Speech pairs to predict POIs for a speech."
#     prompt += "Word limit of 20 words.  The output format should be 1. POI1\n2. POI2\n3. POI3"
#     prompt += "# Few shot prompting: "
#     for index, row in data.iterrows():
#         prompt = " Speech: "+row['speech']+" POI: "+row['poi']+"Country: "+row['country_of_speech']
#     prompt += "\n\n\n\n Now predict the POI for this speech: " + speech
#     prompt += "With this country: " + country
#     prompt += "Know that when predicting a POI, I only want you to give the POI and nothing else. The POI should be in the style of the dataset that I provided. Don't include anything specific to this commitee, such as things about russia when there is nothing about russia in the speech. Just match the style and the general patterns of the POI. Give me 3 POIs. "
#     # Chain of thought
#     prompt += chain_of_thought
    
#     response = client.chat.completions.create(
#         model=model,
#         messages=[
#             {"role": "user", "content": prompt}
#         ]
#     )
#     print(response.choices[0].message.content)

# def make_poi_zero_chain(speech, model, country):
#     prompt = "Generate POIs for the MUN speech. Give me only the actual POI and nothing else for the output. Give me 3 POIs. "
#     prompt += "Word limit of 20 words. The output format should be 1. POI1\n2. POI2\n3. POI3"
#     prompt += "Give me the POI for this speech: " + speech
#     prompt += "With this country: " + country
#     # Chain of thought
#     prompt += chain_of_thought
#     response = client.chat.completions.create(
#         model=model,
#         messages=[
#             {"role": "user", "content": prompt}
#         ]
#     )
    
#     print(response.choices[0].message.content)

## General POI func

# Japanese: 
まず、マークダウン化された、Few Shot Prompting を使っています。Chain of Thought Promptingもしました。鵜飼さんからのフィードバックを元に、フォーマットをより明確にして、新しいPrompting手法（Reflexion & Refinement）を使いました。その結果、Refinement ＋　Few Shot ＋　Chain of Thought の　Promptingで生成されたPOIが一番質が良かったです。鵜飼さんのAPIキーでｏ３とｏ４も試しましたが、4.1で生成されたPOIの方が、質が良かったです。

また、一番新しいバージョンではSelf-consistency promptingとGuardrail promptingを使って検証しました。

Promptingを工夫したらPOIの質が大きく上がったため、Fine Tuningは必要なくPromptingだけで適切なPOIができると考えました。

# English

I made a function that utilizes various prompting techniques to increase the accuracy of the generated POI:
* fewshot prompting
* chain of thought prompting
* refinement prompting
* reflexion prompting
* self-consistency prompting
* guardrail prompting

In [5]:
def make_poi(speech, model, country, fewshot, cot, ref, refl=False, scp=False, guardrail=False):
    # Fewshot = Few shot prompting
    # Cot = Chain of Thought prompting
    # Ref = Refinement prompting
    # Refl = Reflexion prompting
    system = (
        "You are a helpful assistant that predicts Points of Information (POIs) for UN-style speeches. "
        "Each POI should be concise and match the tone and format of prior examples. "
        "Do not include country-specific assumptions unless explicitly stated in the speech."
    )
    prompt = ""

    # Changed Few Shot Engineering to be in markdown and have a clearer format
    if fewshot:
        prompt += "### Few-shot Examples:\n\n"
        for index, row in data.iterrows():
            prompt += (
                f"Example {index + 1}:\n"
                f"Speech:\n{row['speech']}\n"
                f"Country: {row['country_of_speech']}\n"
                f"POI:\n{row['poi']}\n"
                "---\n")

    if cot:
        prompt += "\n###Chain of thought: \n"
        prompt += chain_of_thought + "\n\n\n"
    if guardrail:
        prompt += """
        Avoid POIs that:
        - Use personal attacks
        - Are overly general
        - Rely on assumptions not in the speech
        - Ask multiple questions
        """
    if scp:
        prompt += "Generate 10 POIs and select the 3 POIs that are most specific to the speech content, assertive, and relevant from a diplomatic perspective."
    prompt += "Word limit of 20 words.  The output format should be 1. POI1\n2. POI2\n3. POI3"
    prompt += "The POI should be aggressive. Don't use personal pronouns such as 'you' and instead say 'the delegate'"
    prompt += "Now predict the POI for this speech: " + speech
    prompt += "With this country: " + country

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system},
            {"role": "user", "content": prompt}
        ]
    )

    if ref:
        refine_prompt = (
            "Please revise the following POIs to make them more assertive and more specific to details in the speech. "
            "Avoid general statements. Focus on one clear criticism per POI. Word limit of 20 words. \n\n"
            f"Original POIs:\n{response.choices[0].message.content}\n\n"
            "Revised POIs (same format):"
        )
        if refl:
            refine_prompt = (
                "First, point out the flaws of the POI, such as specificity or whether it is plausible in a high school MUN conference. "
                "Awcknoledging those flaws, create a new POI with a stronger political bias. The output should only be the 3 revised POIs like such: 1. POI1\n2. POI2\n3. POI3. \n\n"
                f"Original POIs:\n{response.choices[0].message.content}\n\n"
                "Revised POIs (same format):"
            )
        refine_response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system},
                {"role": "user", "content": refine_prompt}
            ]
        )
        print(refine_response.choices[0].message.content)
    else:
        print(response.choices[0].message.content)

# Testing

In [6]:
text = "Free market is an economic system based purely on supply and demand. This quintessential system however is used as a disguise to hide corruption and crony capilaism. It is used to hide a system where companies benefit not from free enterprise but from their relation with the government. Under this disguise of economic liberalization, governments privatize their companies unfairly with low prices, exclusive contracts, and regulatory capture. Corruption in privatization processes undermines economic stability, erodes public trust, and distorts the principles of free market economies. During privatization, government officials may be compelled to give unfair advantages to companies beneficial for themselves. By promoting stock holding disclosure and facilitating random audits, we can improve transparency within nations undergoing significant change. Promoting transparency in crucial in ensuring that privatization happens fairly. Transparency in privatization isn’t just a moral imperative; it is essential in attracting foreign investment and ensuring sustainable economic growth. A vote for this directive is a vote to uphold the values of free market during this critical period of economic transformation"

In [7]:
for model in ["gpt-4o", "gpt-4.1"]:
    print(model + ": \n\n\n\n")
    print("Self-consistency and guardrail prompting: ")
    make_poi(text, model, 'Estonia', fewshot=True, cot=True, ref=True, refl=True, scp=False, guardrail=False)

gpt-4o: 




Self-consistency and guardrail prompting: 
1. Considering the entrenched interests benefiting from privatization, how does the delegate propose achieving transparency without undermining the political influence necessary for such economic reforms?
2. With Estonia aiming for greater investment, how can audits address potential investor fears without discouraging essential governmental oversight that ensures equitable business practices?
3. If privatization delays structural reforms, isn't there a risk that political motivations will hinder market efficiency and favor private monopolies over public welfare?
gpt-4.1: 




Self-consistency and guardrail prompting: 
1. If the delegate supports heavy-handed disclosure and audit policies, how does the delegate justify placing regulatory obstacles above the urgent need for economic growth and recovery?
2. By insinuating widespread corruption in privatization, is the delegate not unfairly delegitimizing the efforts of developing ec